In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from random import gauss, seed
from scipy import stats
import seaborn as sns
from mpl_toolkits import mplot3d
import sklearn.metrics as metrics
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

%matplotlib inline

from sklearn.utils import resample
from sklearn.datasets import load_breast_cancer, load_iris, make_classification
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import confusion_matrix, plot_confusion_matrix,\
    precision_score, recall_score, accuracy_score, f1_score, log_loss,\
    roc_curve, roc_auc_score, classification_report


In [2]:
crashes_df = pd.read_csv('data/localdata/Crashes.csv')
people_df = pd.read_csv('data/localdata/People.csv')
vehicles_df = pd.read_csv('data/localdata/Vehicles.csv')

/Users/robertdaly/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (20,24,25,26,29) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/robertdaly/anaconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (19,21,40,41,42,44,48,49,50,53,55,58,59,61,71) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
people_df.head()

,PERSON_ID,PERSON_TYPE,CRASH_RECORD_ID,RD_NO,VEHICLE_ID,CRASH_DATE,SEAT_NO,CITY,STATE,ZIPCODE,...,EMS_RUN_NO,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,PEDPEDAL_ACTION,PEDPEDAL_VISIBILITY,PEDPEDAL_LOCATION,BAC_RESULT,BAC_RESULT VALUE,CELL_PHONE_USE
0,O749947,DRIVER,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,834816.0,09/28/2019 03:30:00 AM,NaN,CHICAGO,IL,60651,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
1,O871921,DRIVER,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,JD208731,827212.0,04/13/2020 10:50:00 PM,NaN,CHICAGO,IL,60620,...,NaN,NONE,NOT OBSCURED,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
2,O10018,DRIVER,71162af7bf22799b776547132ebf134b5b438dcf3dac6b...,HY484534,9579.0,11/01/2015 05:00:00 AM,NaN,NaN,NaN,NaN,...,NaN,IMPROPER BACKING,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
3,O10038,DRIVER,c21c476e2ccc41af550b5d858d22aaac4ffc88745a1700...,HY484750,9598.0,11/01/2015 08:00:00 AM,NaN,NaN,NaN,NaN,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
4,O10039,DRIVER,eb390a4c8e114c69488f5fb8a097fe629f5a92fd528cf4...,HY484778,9600.0,11/01/2015 10:15:00 AM,NaN,NaN,NaN,NaN,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN


In [4]:
people_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1651814 entries, 0 to 1651813
Data columns (total 30 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   PERSON_ID              1651814 non-null  object 
 1   PERSON_TYPE            1651814 non-null  object 
 2   CRASH_RECORD_ID        1651814 non-null  object 
 3   RD_NO                  1642247 non-null  object 
 4   VEHICLE_ID             1619178 non-null  float64
 5   CRASH_DATE             1651814 non-null  object 
 6   SEAT_NO                334546 non-null   float64
 7   CITY                   1203874 non-null  object 
 8   STATE                  1221019 non-null  object 
 9   ZIPCODE                1102457 non-null  object 
 10  SEX                    1625151 non-null  object 
 11  AGE                    1170060 non-null  float64
 12  DRIVERS_LICENSE_STATE  969159 non-null   object 
 13  DRIVERS_LICENSE_CLASS  814924 non-null   object 
 14  SAFETY_EQUIPMENT  

In [5]:
vehicles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1534825 entries, 0 to 1534824
Data columns (total 72 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   CRASH_UNIT_ID             1534825 non-null  int64  
 1   CRASH_RECORD_ID           1534825 non-null  object 
 2   RD_NO                     1526222 non-null  object 
 3   CRASH_DATE                1534825 non-null  object 
 4   UNIT_NO                   1534825 non-null  int64  
 5   UNIT_TYPE                 1532776 non-null  object 
 6   NUM_PASSENGERS            227298 non-null   float64
 7   VEHICLE_ID                1499997 non-null  float64
 8   CMRC_VEH_I                28641 non-null    object 
 9   MAKE                      1499992 non-null  object 
 10  MODEL                     1499848 non-null  object 
 11  LIC_PLATE_STATE           1363068 non-null  object 
 12  VEHICLE_YEAR              1257350 non-null  float64
 13  VEHICLE_DEFECT            1

In [6]:
crashes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 752649 entries, 0 to 752648
Data columns (total 49 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   CRASH_RECORD_ID                752649 non-null  object 
 1   RD_NO                          748463 non-null  object 
 2   CRASH_DATE_EST_I               56767 non-null   object 
 3   CRASH_DATE                     752649 non-null  object 
 4   POSTED_SPEED_LIMIT             752649 non-null  int64  
 5   TRAFFIC_CONTROL_DEVICE         752649 non-null  object 
 6   DEVICE_CONDITION               752649 non-null  object 
 7   WEATHER_CONDITION              752649 non-null  object 
 8   LIGHTING_CONDITION             752649 non-null  object 
 9   FIRST_CRASH_TYPE               752649 non-null  object 
 10  TRAFFICWAY_TYPE                752649 non-null  object 
 11  LANE_CNT                       199004 non-null  float64
 12  ALIGNMENT                     

In [7]:
crashes_df['PRIM_CONTRIBUTORY_CAUSE'].value_counts()

UNABLE TO DETERMINE                                                                 291069
FAILING TO YIELD RIGHT-OF-WAY                                                        82318
FOLLOWING TOO CLOSELY                                                                74202
NOT APPLICABLE                                                                       39743
IMPROPER OVERTAKING/PASSING                                                          36900
FAILING TO REDUCE SPEED TO AVOID CRASH                                               31869
IMPROPER BACKING                                                                     30204
IMPROPER LANE USAGE                                                                  27322
IMPROPER TURNING/NO SIGNAL                                                           24934
DRIVING SKILLS/KNOWLEDGE/EXPERIENCE                                                  24871
DISREGARDING TRAFFIC SIGNALS                                                         14619

In [8]:
crashes_df['SEC_CONTRIBUTORY_CAUSE'].value_counts()

NOT APPLICABLE                                                                      307677
UNABLE TO DETERMINE                                                                 273021
FAILING TO REDUCE SPEED TO AVOID CRASH                                               28574
FAILING TO YIELD RIGHT-OF-WAY                                                        23223
DRIVING SKILLS/KNOWLEDGE/EXPERIENCE                                                  23049
FOLLOWING TOO CLOSELY                                                                19924
IMPROPER OVERTAKING/PASSING                                                          11443
IMPROPER LANE USAGE                                                                  10637
WEATHER                                                                               8730
IMPROPER TURNING/NO SIGNAL                                                            7594
IMPROPER BACKING                                                                      6109

In [9]:
crashes_df['DAMAGE'].value_counts()

OVER $1,500      461672
$501 - $1,500    203738
$500 OR LESS      87239
Name: DAMAGE, dtype: int64

In [10]:
crashes_df['INJURIES_FATAL'].value_counts()

0.0    750205
1.0       756
2.0        51
3.0         8
4.0         1
Name: INJURIES_FATAL, dtype: int64

In [11]:
crashes_df['NOT_RIGHT_OF_WAY_I'].value_counts()

Y    31875
N     3160
Name: NOT_RIGHT_OF_WAY_I, dtype: int64

In [12]:
crashes_df['CRASH_DAY_OF_WEEK'].value_counts()

6    122508
7    111587
5    108214
3    107099
4    106437
2    103537
1     93267
Name: CRASH_DAY_OF_WEEK, dtype: int64

In [13]:
crashes_df['INTERSECTION_RELATED_I'].value_counts()

Y    164328
N      8123
Name: INTERSECTION_RELATED_I, dtype: int64

In [14]:
crashes_df['TRAFFICWAY_TYPE'].value_counts()

NOT DIVIDED                        329054
DIVIDED - W/MEDIAN (NOT RAISED)    122054
ONE-WAY                             96819
PARKING LOT                         51620
DIVIDED - W/MEDIAN BARRIER          43370
FOUR WAY                            43080
OTHER                               20748
ALLEY                               12473
T-INTERSECTION                       8812
UNKNOWN                              8598
CENTER TURN LANE                     5538
DRIVEWAY                             2469
RAMP                                 2363
UNKNOWN INTERSECTION TYPE            2105
FIVE POINT, OR MORE                   973
Y-INTERSECTION                        953
TRAFFIC ROUTE                         788
NOT REPORTED                          472
ROUNDABOUT                            217
L-INTERSECTION                        143
Name: TRAFFICWAY_TYPE, dtype: int64

In [15]:
crashes_df['LANE_CNT'].value_counts()

2.0          91152
4.0          49588
1.0          32547
3.0           8674
0.0           8032
6.0           4502
5.0           1940
8.0           1908
7.0            184
10.0           162
99.0           108
9.0             66
11.0            30
12.0            29
20.0            15
22.0            13
15.0             7
16.0             7
14.0             5
30.0             5
40.0             4
60.0             3
21.0             3
100.0            2
25.0             2
44.0             1
41.0             1
433634.0         1
400.0            1
35.0             1
28.0             1
19.0             1
45.0             1
299679.0         1
17.0             1
1191625.0        1
13.0             1
218474.0         1
80.0             1
902.0            1
24.0             1
Name: LANE_CNT, dtype: int64

In [16]:
crashes_df['ALIGNMENT'].value_counts()

STRAIGHT AND LEVEL       734043
STRAIGHT ON GRADE          9599
CURVE, LEVEL               5494
STRAIGHT ON HILLCREST      2043
CURVE ON GRADE             1125
CURVE ON HILLCREST          345
Name: ALIGNMENT, dtype: int64

In [17]:
crashes_df['ROADWAY_SURFACE_COND'].value_counts()

DRY                559163
WET                 98999
UNKNOWN             60639
SNOW OR SLUSH       26458
ICE                  5214
OTHER                1890
SAND, MUD, DIRT       286
Name: ROADWAY_SURFACE_COND, dtype: int64

In [18]:
crashes_df['ROAD_DEFECT'].value_counts()

NO DEFECTS           610297
UNKNOWN              127257
RUT, HOLES             5857
OTHER                  4159
WORN SURFACE           3087
SHOULDER DEFECT        1409
DEBRIS ON ROADWAY       583
Name: ROAD_DEFECT, dtype: int64

In [19]:
crashes_df['CRASH_TYPE'].value_counts()

NO INJURY / DRIVE AWAY              553291
INJURY AND / OR TOW DUE TO CRASH    199358
Name: CRASH_TYPE, dtype: int64

In [20]:
crashes_df['CRASH_MONTH'].value_counts()

7     68919
6     67542
10    66703
8     66612
5     66519
9     62801
12    61713
11    60487
3     58885
1     57832
2     57478
4     57158
Name: CRASH_MONTH, dtype: int64

In [21]:
crashes_df.drop(columns=['WORK_ZONE_I', 'WORK_ZONE_TYPE', 'DOORING_I', 'WORKERS_PRESENT_I',
                                          'PHOTOS_TAKEN_I', 'STATEMENTS_TAKEN_I', 'RD_NO'], axis=1, inplace=True)


In [22]:
crashes_df.dropna(subset = ['INJURIES_TOTAL', 'LATITUDE', 'MOST_SEVERE_INJURY', 'STREET_DIRECTION', 'BEAT_OF_OCCURRENCE'] , axis=0, inplace=True)

In [23]:
crashes_df = crashes_df[(crashes_df['LANE_CNT'] <= 10)]

In [24]:
crashes_df['NUM_UNITS'].value_counts()

2     175169
3      10259
1       9563
4       1890
5        452
6        116
7         36
8         18
9          7
10         4
11         2
18         1
12         1
Name: NUM_UNITS, dtype: int64

In [25]:
vehicles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1534825 entries, 0 to 1534824
Data columns (total 72 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   CRASH_UNIT_ID             1534825 non-null  int64  
 1   CRASH_RECORD_ID           1534825 non-null  object 
 2   RD_NO                     1526222 non-null  object 
 3   CRASH_DATE                1534825 non-null  object 
 4   UNIT_NO                   1534825 non-null  int64  
 5   UNIT_TYPE                 1532776 non-null  object 
 6   NUM_PASSENGERS            227298 non-null   float64
 7   VEHICLE_ID                1499997 non-null  float64
 8   CMRC_VEH_I                28641 non-null    object 
 9   MAKE                      1499992 non-null  object 
 10  MODEL                     1499848 non-null  object 
 11  LIC_PLATE_STATE           1363068 non-null  object 
 12  VEHICLE_YEAR              1257350 non-null  float64
 13  VEHICLE_DEFECT            1

In [26]:
vehicles_df['VEHICLE_YEAR'].value_counts()

2015.0    94512
2016.0    88892
2017.0    87467
2014.0    83412
2013.0    77084
          ...  
2920.0        1
2929.0        1
2992.0        1
2994.0        1
8324.0        1
Name: VEHICLE_YEAR, Length: 221, dtype: int64

In [27]:
vehicles_df['MANEUVER'].value_counts()

STRAIGHT AHEAD                        686948
PARKED                                206841
UNKNOWN/NA                            116539
SLOW/STOP IN TRAFFIC                  112927
TURNING LEFT                           88508
BACKING                                61476
TURNING RIGHT                          49283
PASSING/OVERTAKING                     36223
CHANGING LANES                         28903
OTHER                                  24992
ENTERING TRAFFIC LANE FROM PARKING     17639
MERGING                                10374
STARTING IN TRAFFIC                     8672
U-TURN                                  8474
LEAVING TRAFFIC LANE TO PARK            7212
AVOIDING VEHICLES/OBJECTS               6448
SKIDDING/CONTROL LOSS                   5898
ENTER FROM DRIVE/ALLEY                  5668
PARKED IN TRAFFIC LANE                  4605
SLOW/STOP - LEFT TURN                   3042
DRIVING WRONG WAY                       2191
NEGOTIATING A CURVE                     1955
SLOW/STOP 

In [28]:
crashes_df['WEATHER_CONDITION'].value_counts()

CLEAR                     155668
RAIN                       18279
SNOW                        7889
UNKNOWN                     7877
CLOUDY/OVERCAST             6332
OTHER                        658
FOG/SMOKE/HAZE               418
SLEET/HAIL                   353
SEVERE CROSS WIND GATE        43
FREEZING RAIN/DRIZZLE          1
Name: WEATHER_CONDITION, dtype: int64

In [29]:
crashes_df['LIGHTING_CONDITION'].value_counts()

DAYLIGHT                  127402
DARKNESS, LIGHTED ROAD     44145
DARKNESS                    9875
UNKNOWN                     6720
DUSK                        6147
DAWN                        3229
Name: LIGHTING_CONDITION, dtype: int64

In [30]:
vehicles_df['AREA_00_I'].value_counts()

Y    46998
N     6467
Name: AREA_00_I, dtype: int64

In [31]:
vehicles_df.columns = vehicles_df.columns.str.title()
vehicles_df.columns = vehicles_df.columns.str.replace('_', ' ')
vehicles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1534825 entries, 0 to 1534824
Data columns (total 72 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   Crash Unit Id             1534825 non-null  int64  
 1   Crash Record Id           1534825 non-null  object 
 2   Rd No                     1526222 non-null  object 
 3   Crash Date                1534825 non-null  object 
 4   Unit No                   1534825 non-null  int64  
 5   Unit Type                 1532776 non-null  object 
 6   Num Passengers            227298 non-null   float64
 7   Vehicle Id                1499997 non-null  float64
 8   Cmrc Veh I                28641 non-null    object 
 9   Make                      1499992 non-null  object 
 10  Model                     1499848 non-null  object 
 11  Lic Plate State           1363068 non-null  object 
 12  Vehicle Year              1257350 non-null  float64
 13  Vehicle Defect            1

In [32]:
area_columns = vehicles_df.columns[vehicles_df.columns.str.contains('Area', case=False)]
area_df = vehicles_df[area_columns]


In [33]:
area_df.describe()

,Area 00 I,Area 01 I,Area 02 I,Area 03 I,Area 04 I,Area 05 I,Area 06 I,Area 07 I,Area 08 I,Area 09 I,Area 10 I,Area 11 I,Area 12 I,Area 99 I
count,53465,407893,244949,146371,146938,228059,239041,218892,223387,97955,141966,276303,271157,171686
unique,2,2,2,2,2,2,2,2,2,2,2,2,2,2
top,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y
freq,46998,389685,233249,139699,140369,219196,229759,209606,211904,93180,134739,260396,260112,165301


In [34]:
vehicles_df['Exceed Speed Limit I'].value_counts()

Y    1799
N     598
Name: Exceed Speed Limit I, dtype: int64

In [35]:
vehicles_df['Travel Direction'].value_counts()

N          352860
S          345009
W          312220
E          305982
UNKNOWN    119129
SE          18981
NW          17327
SW          14320
NE          14169
Name: Travel Direction, dtype: int64

In [36]:
vehicles_df['Carrier Name'].value_counts()

CHICAGO TRANSIT AUTHORITY       560
CTA                             276
UNKNOWN                         186
CITY OF CHICAGO                 134
JB HUNT                         131
                               ... 
MLEWIS TRANSPORT INC              1
VCNA PRAIRIE INC                  1
MIDLAND PAPER COMPANY             1
AMBULANCE TRANSPORTATION INC      1
UNITED PIPE & STEEL CORP          1
Name: Carrier Name, Length: 9787, dtype: int64

In [37]:
vehicles_df['Hazmat Report I'].value_counts()

N    9660
U    1572
Y      12
Name: Hazmat Report I, dtype: int64

In [38]:
vehicles_df['Hazmat Vio Cause Crash I'].value_counts()

N    10355
U     1039
Y       13
Name: Hazmat Vio Cause Crash I, dtype: int64

In [39]:
crashes_df['INTERSECTION_RELATED_I'].value_counts()

Y    44464
N     2067
Name: INTERSECTION_RELATED_I, dtype: int64

In [40]:
crashes_df['INJURIES_TOTAL'].value_counts()

0.0     172828
1.0      18801
2.0       3976
3.0       1188
4.0        416
5.0        175
6.0         74
7.0         28
8.0         14
9.0          9
11.0         3
10.0         3
13.0         1
21.0         1
15.0         1
Name: INJURIES_TOTAL, dtype: int64

In [41]:
vehicles_df['Occupant Cnt'].value_counts()

1.0     1085719
0.0      199172
2.0      151689
3.0       39841
4.0       15786
5.0        5189
6.0        1400
7.0         541
8.0         207
9.0          99
11.0         65
10.0         61
12.0         41
13.0         36
15.0         18
14.0         16
16.0         13
20.0         11
18.0         11
19.0         10
17.0          7
26.0          6
28.0          6
36.0          5
29.0          4
44.0          4
21.0          4
33.0          3
35.0          3
22.0          3
27.0          3
25.0          3
30.0          3
43.0          2
41.0          2
39.0          2
99.0          2
34.0          2
60.0          1
32.0          1
47.0          1
38.0          1
37.0          1
24.0          1
23.0          1
31.0          1
Name: Occupant Cnt, dtype: int64

In [42]:
people_df['SEAT_NO'].value_counts()

3.0     161940
6.0      56306
4.0      44608
5.0      15761
1.0      14747
2.0      13897
12.0      8730
7.0       7786
10.0      7453
11.0      2928
8.0        390
Name: SEAT_NO, dtype: int64

In [43]:
people_df['AGE'].describe()

count    1.170060e+06
mean     3.785983e+01
std      1.709937e+01
min     -1.770000e+02
25%      2.500000e+01
50%      3.500000e+01
75%      5.000000e+01
max      1.100000e+02
Name: AGE, dtype: float64

In [44]:
people_df['INJURY_CLASSIFICATION'].value_counts()

NO INDICATION OF INJURY     1509642
NONINCAPACITATING INJURY      79984
REPORTED, NOT EVIDENT         45390
INCAPACITATING INJURY         15200
FATAL                           907
Name: INJURY_CLASSIFICATION, dtype: int64

In [45]:
people_df['DRIVER_ACTION'].value_counts()

NONE                                 472168
UNKNOWN                              327906
FAILED TO YIELD                      119556
OTHER                                116877
FOLLOWED TOO CLOSELY                  79188
IMPROPER BACKING                      39677
IMPROPER TURN                         34433
IMPROPER LANE CHANGE                  33854
IMPROPER PASSING                      29239
DISREGARDED CONTROL DEVICES           22998
TOO FAST FOR CONDITIONS               20056
WRONG WAY/SIDE                         5208
IMPROPER PARKING                       4888
OVERCORRECTED                          2253
EVADING POLICE VEHICLE                 2080
CELL PHONE USE OTHER THAN TEXTING      1988
EMERGENCY VEHICLE ON CALL              1199
TEXTING                                 531
STOPPED SCHOOL BUS                      155
LICENSE RESTRICTIONS                     54
Name: DRIVER_ACTION, dtype: int64

In [46]:
people_df['DRIVER_VISION'].value_counts()

NOT OBSCURED              674045
UNKNOWN                   607634
OTHER                      13223
MOVING VEHICLES             7596
PARKED VEHICLES             4669
WINDSHIELD (WATER/ICE)      3651
BLINDED - SUNLIGHT          1540
TREES, PLANTS                558
BUILDINGS                    476
BLINDED - HEADLIGHTS         129
HILLCREST                     95
BLOWING MATERIALS             94
EMBANKMENT                    78
SIGNBOARD                     34
Name: DRIVER_VISION, dtype: int64

In [47]:
people_df['PHYSICAL_CONDITION'].value_counts()

NORMAL                          860433
UNKNOWN                         430416
IMPAIRED - ALCOHOL                5773
REMOVED BY EMS                    4853
OTHER                             3852
FATIGUED/ASLEEP                   3500
EMOTIONAL                         2972
ILLNESS/FAINTED                   1226
HAD BEEN DRINKING                  983
IMPAIRED - DRUGS                   675
IMPAIRED - ALCOHOL AND DRUGS       360
MEDICATED                          168
Name: PHYSICAL_CONDITION, dtype: int64

In [48]:
people_df['BAC_RESULT'].value_counts()

TEST NOT OFFERED                   1295757
TEST REFUSED                         13901
TEST PERFORMED, RESULTS UNKNOWN       3258
TEST TAKEN                            2385
Name: BAC_RESULT, dtype: int64

In [49]:
people_df['PEDPEDAL_LOCATION'].value_counts()

IN ROADWAY         13838
IN CROSSWALK       10257
UNKNOWN/NA          2605
NOT IN ROADWAY      1387
BIKEWAY             1363
BIKE LANE            945
DRIVEWAY ACCESS      438
SHOULDER             165
Name: PEDPEDAL_LOCATION, dtype: int64

In [50]:
people_df['PEDPEDAL_ACTION'].value_counts()

CROSSING - WITH SIGNAL                               6280
WITH TRAFFIC                                         4812
UNKNOWN/NA                                           3996
OTHER ACTION                                         3718
CROSSING - NO CONTROLS (NOT AT INTERSECTION)         1896
NO ACTION                                            1670
CROSSING - NO CONTROLS (AT INTERSECTION)             1606
CROSSING - AGAINST SIGNAL                            1477
NOT AT INTERSECTION                                  1040
CROSSING - CONTROLS PRESENT (NOT AT INTERSECTION)     951
AGAINST TRAFFIC                                       937
STANDING IN ROADWAY                                   731
TURNING LEFT                                          446
PARKED VEHICLE                                        360
ENTER FROM DRIVE/ALLEY                                298
WORKING IN ROADWAY                                    214
TURNING RIGHT                                         197
INTOXICATED PE

In [51]:
crashes_df['INJURIES_TOTAL'].value_counts()

0.0     172828
1.0      18801
2.0       3976
3.0       1188
4.0        416
5.0        175
6.0         74
7.0         28
8.0         14
9.0          9
11.0         3
10.0         3
13.0         1
21.0         1
15.0         1
Name: INJURIES_TOTAL, dtype: int64

In [52]:
people_df['ZIPCODE'].value_counts()

60629         40500
60639         34208
60620         30601
60617         29474
60623         28552
              ...  
11227             1
06515-2324        1
387740000         1
71646             1
69429             1
Name: ZIPCODE, Length: 13784, dtype: int64

In [53]:
people_df['AIRBAG_DEPLOYED'].value_counts()

DID NOT DEPLOY                            865492
DEPLOYMENT UNKNOWN                        327866
NOT APPLICABLE                            319806
DEPLOYED, FRONT                            52222
DEPLOYED, COMBINATION                      39591
DEPLOYED, SIDE                             14673
DEPLOYED OTHER (KNEE, AIR, BELT, ETC.)       786
Name: AIRBAG_DEPLOYED, dtype: int64

In [54]:
people_df['SAFETY_EQUIPMENT'].value_counts()

SAFETY BELT USED                              789946
USAGE UNKNOWN                                 764325
NONE PRESENT                                   52929
SAFETY BELT NOT USED                            9203
CHILD RESTRAINT USED                            7822
HELMET NOT USED                                 7410
CHILD RESTRAINT - FORWARD FACING                3850
BICYCLE HELMET (PEDACYCLIST INVOLVED ONLY)      2846
CHILD RESTRAINT - TYPE UNKNOWN                  2006
CHILD RESTRAINT - REAR FACING                   1816
HELMET USED                                     1358
DOT COMPLIANT MOTORCYCLE HELMET                 1150
BOOSTER SEAT                                    1030
CHILD RESTRAINT NOT USED                         734
SHOULD/LAP BELT USED IMPROPERLY                  214
NOT DOT COMPLIANT MOTORCYCLE HELMET              206
WHEELCHAIR                                       185
CHILD RESTRAINT USED IMPROPERLY                  132
STRETCHER                                     

In [55]:
people_df['EJECTION'].value_counts()

NONE                  1527825
UNKNOWN                 96468
TOTALLY EJECTED          5090
PARTIALLY EJECTED        1283
TRAPPED/EXTRICATED        970
Name: EJECTION, dtype: int64

In [56]:
people_df['HOSPITAL'].value_counts()

REFUSED                                      93707
DNA                                          35354
NONE                                         20209
99                                            7207
DECLINED                                      6179
                                             ...  
MECERY                                           1
LAWNDALE CHRISTIAN HEALTH CENTER FOLLOWUP        1
ST JJOSEPH                                       1
EMS 47 AND CFD 124 ON SCENE                      1
EMS REFUSED BY DRIVER                            1
Name: HOSPITAL, Length: 6934, dtype: int64

In [57]:
vehicles_df['Vehicle Use'].value_counts()

PERSONAL                        969323
UNKNOWN/NA                      304342
NOT IN USE                       77860
OTHER                            47092
TAXI/FOR HIRE                    19360
COMMERCIAL - SINGLE UNIT         18359
RIDESHARE SERVICE                12466
CTA                              10253
POLICE                            9790
CONSTRUCTION/MAINTENANCE          6755
COMMERCIAL - MULTI-UNIT           6117
SCHOOL BUS                        4008
OTHER TRANSIT                     3964
TOW TRUCK                         2860
AMBULANCE                         1777
FIRE                              1501
STATE OWNED                       1273
DRIVER EDUCATION                  1167
MASS TRANSIT                       832
LAWN CARE/LANDSCAPING              553
AGRICULTURE                        160
CAMPER/RV - SINGLE UNIT             84
MILITARY                            59
HOUSE TRAILER                       24
CAMPER/RV - TOWED/MULTI-UNIT        18
Name: Vehicle Use, dtype:

In [58]:
vehicles_df['Vehicle Defect'].value_counts()

NONE                822690
UNKNOWN             661824
OTHER                 7853
BRAKES                4822
TIRES                  753
STEERING               689
WHEELS                 382
SUSPENSION             254
ENGINE/MOTOR           199
FUEL SYSTEM            190
LIGHTS                  96
WINDOWS                 92
CARGO                   51
SIGNALS                 40
RESTRAINT SYSTEM        23
TRAILER COUPLING        21
EXHAUST                 18
Name: Vehicle Defect, dtype: int64

In [59]:
vehicles_df['Make'].value_counts()

CHEVROLET                                      175191
UNKNOWN                                        150103
FORD                                           149458
NISSAN                                         122598
HONDA                                          109163
                                                ...  
RUSH INDUS., INC., ELKHART, IN                      1
NASH                                                1
VERSATILE POWER CORP., GRANTSBURG WISCONSIN         1
LBT, DIVISION FRUEHAUF CORP.                        1
BATAVIA TRAILERS, INC.                              1
Name: Make, Length: 1293, dtype: int64

In [60]:
vehicles_df

,Crash Unit Id,Crash Record Id,Rd No,Crash Date,Unit No,Unit Type,Num Passengers,Vehicle Id,Cmrc Veh I,Make,...,Trailer1 Length,Trailer2 Length,Total Vehicle Length,Axle Cnt,Vehicle Config,Cargo Body Type,Load Type,Hazmat Out Of Service I,Mcs Out Of Service I,Hazmat Class
0,1554880,91a5d08b2b701f2d37cbb52ecdbeb09579bc7f2ebc60b3...,JG223284,04/14/2023 02:05:00 PM,1,DRIVER,NaN,1478881.0,NaN,FORD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,749947,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,09/28/2019 03:30:00 AM,1,DRIVER,NaN,834816.0,NaN,HONDA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,749949,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,09/28/2019 03:30:00 AM,2,PARKED,NaN,834819.0,NaN,TOYOTA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,749950,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,09/28/2019 03:30:00 AM,3,PARKED,NaN,834817.0,NaN,GENERAL MOTORS CORPORATION (GMC),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1554881,91a5d08b2b701f2d37cbb52ecdbeb09579bc7f2ebc60b3...,JG223284,04/14/2023 02:05:00 PM,2,DRIVER,1.0,1478892.0,NaN,ISUZU,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1534820,1571683,545044224202f24b78f10eef0aef2935ff9abfc4f10309...,JG253335,05/08/2023 11:25:00 PM,2,DRIVER,NaN,1495169.0,NaN,TOYOTA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1534821,1571684,6b53b6a8fb20fc4839856798fb480cd21db71c6a3fa9f9...,JG253334,05/08/2023 11:00:00 AM,1,DRIVER,NaN,1495166.0,NaN,UNKNOWN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1534822,1571685,6b53b6a8fb20fc4839856798fb480cd21db71c6a3fa9f9...,JG253334,05/08/2023 11:00:00 AM,2,DRIVER,NaN,1495168.0,NaN,TOYOTA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1534823,1571686,80405e9e967b0f5f8f3966f7917f389e63c4f5243a862b...,JG252873,05/08/2023 04:15:00 PM,1,DRIVER,NaN,1495170.0,NaN,TOYOTA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
area_columns = vehicles_df.columns[vehicles_df.columns.str.contains('Area', case=False)]

In [62]:
vehicles_df[area_columns] = vehicles_df[area_columns].fillna(0).mask(vehicles_df[area_columns] == 'Y', 'Y')

In [63]:
vehicles_df[area_columns]

,Area 00 I,Area 01 I,Area 02 I,Area 03 I,Area 04 I,Area 05 I,Area 06 I,Area 07 I,Area 08 I,Area 09 I,Area 10 I,Area 11 I,Area 12 I,Area 99 I
0,0,0,0,0,Y,0,0,0,0,0,Y,0,0,0
1,0,Y,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,Y,0,0,0,0
3,0,0,0,0,0,0,0,0,0,Y,0,0,0,0
4,0,0,0,Y,Y,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1534820,0,0,0,N,N,0,0,0,Y,Y,0,0,0,0
1534821,0,0,0,0,0,0,0,0,0,0,0,0,0,Y
1534822,0,0,0,0,0,Y,0,0,0,0,0,0,0,0
1534823,0,0,0,0,0,0,0,0,0,0,Y,Y,0,0


In [ ]:
vehicles_df[area_columns] = vehicles_df[area_columns].replace(to_replace="N", value = 0)
vehicles_df[area_columns] = vehicles_df[area_columns].replace(to_replace="Y", value = 1)

In [ ]:
b

In [ ]:
vehicles_df[area_columns].loc[vehicles_df[area_columns].sum(axis=1) > 1]

In [ ]:
vehicles_df['Num Areas'] = vehicles_df[area_columns].sum(axis=1)

In [ ]:
vehicles_df['Num Areas'].value_counts()

In [ ]:
# Injury Score column is a linear combination of the factors that make up Injuries Total simply 
# multiplied by a constant depending on their severity
crashes_df['Injury Score'] = crashes_df['Injuries Fatal']*7 +
                            crashes_df['Injuries Incapacitating']*3 + 
                            crashes_df['Injuries Non Incapacitating']*2 + 
                            crashes_df['Injuries Reported Not Evident']

In [ ]:
# Injury Score

crashes_df[['Injuries Total', 'Injuries Fatal', 'Injuries Incapacitating', 
            'Injuries Non Incapacitating', 'Injuries Reported Not Evident']][crashes_df['Injuries Total']!=0][:50]

crashes_df['total injured'] = crashes_df['Injuries Fatal'] + crashes_df['Injuries Incapacitating'] + crashes_df['Injuries Non Incapacitating'] + crashes_df['Injuries Reported Not Evident']

# Checking if added up columns in 'total injured' make up 'Injuries Total'
(crashes_df['total injured'] == crashes_df['Injuries Total']).value_counts()

# Injury Score column is a linear combination of the factors that make up Injuries Total simply multiplied by a constant depending on their severity
crashes_df['Injury Score'] = crashes_df['Injuries Fatal']*7 + crashes_df['Injuries Incapacitating']*3 + crashes_df['Injuries Non Incapacitating']*2 + crashes_df['Injuries Reported Not Evident']

# Accounting Material Damage of Car and Surrounding Areas (Crash Score)

crashes_df['Damage'].value_counts()
mapping = {'OVER $1,500': 3, '$501 - $1,500': 2, "$500 OR LESS": 1}
crashes_df['Damage_ODE'] = crashes_df['Damage'].map(mapping)
crashes_df['Injury Score'] = crashes_df['Injury Score'] + crashes_df['Damage_ODE']*3
crashes_df['Crash Score'] = crashes_df['Injury Score']